In [ ]:
# from PIL import Image
# import numpy as np
# import math
# from skimage.metrics import structural_similarity as ssim

# def encode(cover_path, secret_path, stego_path, bits_to_hide=1):
#     """
#     在载体图像中隐藏秘密图像，使用多个最低有效位。

#     参数：
#         cover_path (str): 载体图像的路径。
#         secret_path (str): 秘密图像的路径。
#         stego_path (str): 输出图像的路径。
#         bits_to_hide (int): 要使用的最低有效位数量（1、2、3...）
#     """

#     cover_image = Image.open(cover_path).convert('RGB')
#     secret_image = Image.open(secret_path).convert('RGB')

#     cover_width, cover_height = cover_image.size
#     secret_width, secret_height = secret_image.size

#     # 检查载体图像是否足够大以隐藏秘密图像
#     if secret_width > cover_width or secret_height > cover_height:
#         raise ValueError("载体图像太小，无法隐藏秘密图像。")

#     cover_pixels = cover_image.load()
#     secret_pixels = secret_image.load()

#     # 掩码，用于保留最低有效位
#     mask = (1 << bits_to_hide) - 1  # 生成掩码

#     for x in range(secret_width):
#         for y in range(secret_height):
#             # 获取秘密图像的 RGB 值
#             secret_r, secret_g, secret_b = secret_pixels[x, y]

#             # 获取载体图像的 RGB 值
#             cover_r, cover_g, cover_b = cover_pixels[x, y]

#             # 将秘密图像的高位替换载体图像的低位
#             cover_r = (cover_r & ~(mask)) | ((secret_r >> (8 - bits_to_hide)) & mask)
#             cover_g = (cover_g & ~(mask)) | ((secret_g >> (8 - bits_to_hide)) & mask)
#             cover_b = (cover_b & ~(mask)) | ((secret_b >> (8 - bits_to_hide)) & mask)

#             # 更新载体图像的像素值
#             cover_pixels[x, y] = (cover_r, cover_g, cover_b)

#     # 保存隐藏了秘密图像的载体图像
#     cover_image.save(stego_path)

# def decode(stego_path, extracted_path, secret_width, secret_height, bits_to_extract=1):
#     """
#     从隐藏图像中提取秘密图像，使用多个最低有效位。

#     参数：
#         stego_path (str): 隐藏图像的路径。
#         extracted_path (str): 输出秘密图像的路径。
#         secret_width (int): 秘密图像的宽度。
#         secret_height (int): 秘密图像的高度。
#         bits_to_extract (int): 要提取的最低有效位数量（1、2、3...）
#     """

#     stego_image = Image.open(stego_path).convert('RGB')
#     stego_pixels = stego_image.load()

#     secret_image = Image.new('RGB', (secret_width, secret_height))
#     secret_pixels = secret_image.load()

#     # 掩码，用于提取最低有效位
#     mask = (1 << bits_to_extract) - 1  # 生成掩码

#     for x in range(secret_width):
#         for y in range(secret_height):
#             # 获取隐藏图像的 RGB 值
#             stego_r, stego_g, stego_b = stego_pixels[x, y]

#             # 从隐藏图像的最低有效位提取秘密图像的 RGB 值
#             secret_r = (stego_r & mask) << (8 - bits_to_extract)
#             secret_g = (stego_g & mask) << (8 - bits_to_extract)
#             secret_b = (stego_b & mask) << (8 - bits_to_extract)

#             # 设置秘密图像的像素值
#             secret_pixels[x, y] = (secret_r, secret_g, secret_b)

#     # 保存提取的秘密图像
#     secret_image.save(extracted_path)

from PIL import Image
import numpy as np
import math
from skimage.metrics import structural_similarity as ssim
from methods.lsb import LSBSteganography




cover_path = "./dataset/cover.jpg"  # 载体图像路径
secret_path = "./dataset/secret.jpg"  # 秘密图像路径
stego_path = "./results/stego.png"  # 隐藏图像路径
extracted_path = "./results/extracted.png"  # 提取秘密图像的路径

stego = LSBSteganography(bits_to_hide=2)  # 创建对象

# 隐藏图像
stego.encode(cover_path, secret_path, stego_path)

# 提取图像
secret_width, secret_height = Image.open(secret_path).size
stego.decode(stego_path, extracted_path, secret_width, secret_height)




In [2]:


def psnr(img1, img2):
    """计算两个图像之间的 PSNR"""
    mse = np.mean((np.array(img1) - np.array(img2)) ** 2)
    if mse == 0:
        return float('inf')  # 完全相同的图像
    max_pixel = 255.0
    return 20 * math.log10(max_pixel / math.sqrt(mse))

def compare_images(cover_path, stego_path):
    """对比原始图像和隐藏图像之间的差异"""
    
    cover_image = Image.open(cover_path)
    stego_image = Image.open(stego_path)

    # 计算 PSNR
    psnr_value = psnr(cover_image, stego_image)
    print(f"PSNR between cover and stego image: {psnr_value} dB")

    # 计算 SSIM
    cover_array = np.array(cover_image.convert('RGB'))
    stego_array = np.array(stego_image.convert('RGB'))
    
    ssim_value = ssim(cover_array, stego_array, multichannel=True, win_size=3)
    print(f"SSIM between cover and stego image: {ssim_value}")

# 示例用法
cover_path = "./dataset/cover.jpg"  # 载体图像路径
stego_path = "./results/stego.png"  # 隐藏图像路径

# 对比原始图像和隐藏图像
compare_images(cover_path, stego_path)


PSNR between cover and stego image: 61.09628778334568 dB
SSIM between cover and stego image: 0.9997479615490493
